In [1]:
import numpy as np
import pandas as pd

# Data prepocessing

In [66]:
text_df = pd.read_csv('dataset/multimodal variables - Text-LIWC_Gemini-1.5-pro.csv').sort_values('video')
text_var = text_df[['self-confidence','creativity','enthusiasm','future-orientation','trustworthiness','proactive-personality']]
audio_df = pd.read_csv('dataset/multimodal variables - Audio-Sonix.csv').sort_values('video')
audio_var = audio_df[['assertiveness',	'creativity','enthusiasm',	'future-orientation',	'optimism',	'proactive-personality','self-confidence']]
video_df = pd.read_csv('dataset/multimodal variables - Video-Gemini-1.5-Pro.csv').sort_values('video')
video_var = video_df[['self-confidence',	'creativity','enthusiasm',	'trustworthiness',	'optimism',	'proactive-personality','future-orientation']]
startup_df = pd.read_csv('dataset/Startup_New_backUP.xlsx - Sheet1.csv')
y_var = startup_df[['Fundraised','Backers']]
y_var['Fundraised'] = y_var['Fundraised'].copy().apply(lambda x: x.replace('$','').replace(',','')).astype(float)
y_var['Backers'] = y_var['Backers'].copy().apply(lambda x: x.replace('$','').replace(',','')).astype(float)

/var/folders/t9/x5knz49s7qj_dry4y1r799x00000gn/T/ipykernel_82845/3129778605.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_var['Fundraised'] = y_var['Fundraised'].copy().apply(lambda x: x.replace('$','').replace(',','')).astype(float)
/var/folders/t9/x5knz49s7qj_dry4y1r799x00000gn/T/ipykernel_82845/3129778605.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_var['Backers'] = y_var['Backers'].copy().apply(lambda x: x.replace('$','').replace(',','')).astype(float)


In [67]:
for var_df in [text_var, audio_var, video_var]:
    #impute missing values with mean
    var_df.fillna(var_df.mean(), inplace=True)

/var/folders/t9/x5knz49s7qj_dry4y1r799x00000gn/T/ipykernel_82845/1621441885.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.fillna(var_df.mean(), inplace=True)


In [68]:
video_var

,self-confidence,creativity,enthusiasm,trustworthiness,optimism,proactive-personality,future-orientation
0,10.0,9.0,8.5,10.0,9.5,9.000000,10.0
1,8.5,9.5,9.5,9.5,9.0,9.500000,9.5
2,9.0,9.5,9.0,10.0,10.0,8.985129,9.0
3,9.0,8.5,9.0,10.0,9.0,9.000000,10.0
4,10.0,9.5,9.0,9.5,9.5,9.000000,9.0
...,...,...,...,...,...,...,...
112,8.0,8.5,9.5,8.0,10.0,8.000000,9.5
113,9.5,9.0,4.0,10.0,10.0,10.000000,9.5
114,10.0,8.0,9.0,10.0,10.0,10.000000,9.5
115,9.5,9.0,10.0,8.0,9.5,9.700000,9.0


# Model 1(Text Only)

In [69]:
import statsmodels.api as sm

X = pd.concat([text_var], axis=1)
X = sm.add_constant(X)
y_fundraised = y_var['Fundraised']

model = sm.OLS(y_fundraised, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Fundraised   R-squared:                       0.076
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     1.513
Date:                Wed, 22 Jan 2025   Prob (F-statistic):              0.181
Time:                        22:07:55   Log-Likelihood:                -1341.2
No. Observations:                 117   AIC:                             2696.
Df Residuals:                     110   BIC:                             2716.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                  1.924e+04   2.29e+04      0.840      0.403   -2.62e+04    6.47e+04
self-confidence           6.7892     90.055      0.075      0.940    -171.679     185.258
creativity             3874.0730   1607.972      2.409      0.018     687.451    7060.695
enthusiasm             -233.1434    469.810     -0.496      0.621   -1164.196     697.909
future-orientation    -3622.6905   3048.007     -1.189      0.237   -9663.124    2417.743
trustworthiness       -1577.5434   1213.231     -1.300      0.196   -3981.883     826.796
proactive-personality -1245.3370   2434.990     -0.511      0.610   -6070.916    3580.242
==============================================================================
Omnibus:                      122.384   Durbin-Watson:                   1.772
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1979.058
Skew:                           3.616   Prob(JB):                         0.00
Kurtosis:                      21.806   Cond. No.                         862.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [70]:
y_backers = y_var['Backers']
model = sm.OLS(y_backers, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Backers   R-squared:                       0.054
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.055
Date:                Wed, 22 Jan 2025   Prob (F-statistic):              0.394
Time:                        22:07:56   Log-Likelihood:                -841.31
No. Observations:                 117   AIC:                             1697.
Df Residuals:                     110   BIC:                             1716.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   186.4009    319.509      0.583      0.561    -446.790     819.592
self-confidence           0.3355      1.256      0.267      0.790      -2.153       2.824
creativity               32.1740     22.423      1.435      0.154     -12.264      76.612
enthusiasm               -4.9199      6.552     -0.751      0.454     -17.904       8.064
future-orientation      -72.8670     42.505     -1.714      0.089    -157.102      11.368
trustworthiness         -10.7113     16.919     -0.633      0.528     -44.240      22.818
proactive-personality     5.2913     33.956      0.156      0.876     -62.002      72.585
==============================================================================
Omnibus:                       47.586   Durbin-Watson:                   1.524
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               96.922
Skew:                           1.750   Prob(JB):                     8.99e-22
Kurtosis:                       5.761   Cond. No.                         862.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Model2 (Text+Audio)

In [71]:
X = pd.concat([text_var[['creativity','future-orientation']], audio_var[['self-confidence','enthusiasm']],text_var[['trustworthiness','proactive-personality']]], axis=1)
X = sm.add_constant(X)
y_fundraised = y_var['Fundraised']

model = sm.OLS(y_fundraised, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Fundraised   R-squared:                       0.108
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     2.209
Date:                Wed, 22 Jan 2025   Prob (F-statistic):             0.0473
Time:                        22:07:58   Log-Likelihood:                -1339.2
No. Observations:                 117   AIC:                             2692.
Df Residuals:                     110   BIC:                             2712.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                  1.651e+04   2.83e+04      0.583      0.561   -3.96e+04    7.27e+04
creativity             2394.5678   1771.258      1.352      0.179   -1115.650    5904.785
future-orientation    -3585.8829   3005.635     -1.193      0.235   -9542.345    2370.580
self-confidence        5640.8536   2813.671      2.005      0.047      64.818    1.12e+04
enthusiasm            -4032.2809   2929.106     -1.377      0.171   -9837.082    1772.520
trustworthiness       -1526.3868   1186.422     -1.287      0.201   -3877.597     824.823
proactive-personality -1222.2715   2389.687     -0.511      0.610   -5958.070    3513.527
==============================================================================
Omnibus:                      118.908   Durbin-Watson:                   1.707
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1879.417
Skew:                           3.464   Prob(JB):                         0.00
Kurtosis:                      21.372   Cond. No.                         230.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [72]:
y_backers = y_var['Backers']
model = sm.OLS(y_backers, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Backers   R-squared:                       0.053
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.021
Date:                Wed, 22 Jan 2025   Prob (F-statistic):              0.416
Time:                        22:08:00   Log-Likelihood:                -841.42
No. Observations:                 117   AIC:                             1697.
Df Residuals:                     110   BIC:                             1716.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   239.8386    402.378      0.596      0.552    -557.580    1037.257
creativity               25.7891     25.152      1.025      0.307     -24.057      75.635
future-orientation      -68.9854     42.681     -1.616      0.109    -153.569      15.598
self-confidence          21.7230     39.955      0.544      0.588     -57.459     100.905
enthusiasm              -23.8297     41.594     -0.573      0.568    -106.260      58.600
trustworthiness         -11.7504     16.848     -0.697      0.487     -45.138      21.638
proactive-personality     5.5590     33.934      0.164      0.870     -61.691      72.809
==============================================================================
Omnibus:                       49.626   Durbin-Watson:                   1.517
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              106.519
Skew:                           1.791   Prob(JB):                     7.41e-24
Kurtosis:                       6.004   Cond. No.                         230.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Model3 (Text+Audio+Video)

In [77]:
X = pd.concat([video_var[['creativity','future-orientation']], audio_var[['self-confidence','enthusiasm']],text_var[['proactive-personality','trustworthiness']]], axis=1)
X = sm.add_constant(X)
y_fundraised = y_var['Fundraised']

model = sm.OLS(y_fundraised, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Fundraised   R-squared:                       0.087
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     1.756
Date:                Wed, 22 Jan 2025   Prob (F-statistic):              0.115
Time:                        22:09:08   Log-Likelihood:                -1340.5
No. Observations:                 117   AIC:                             2695.
Df Residuals:                     110   BIC:                             2714.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -6042.2403   3.37e+04     -0.179      0.858   -7.29e+04    6.08e+04
creativity             1734.4685   1835.204      0.945      0.347   -1902.476    5371.413
future-orientation      941.3499   1784.619      0.527      0.599   -2595.346    4478.045
self-confidence        7241.8159   2603.754      2.781      0.006    2081.788    1.24e+04
enthusiasm            -4896.7297   3008.178     -1.628      0.106   -1.09e+04    1064.773
proactive-personality  -525.5407   2307.042     -0.228      0.820   -5097.557    4046.476
trustworthiness       -1556.9181   1189.857     -1.308      0.193   -3914.936     801.099
==============================================================================
Omnibus:                      114.367   Durbin-Watson:                   1.726
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1606.237
Skew:                           3.322   Prob(JB):                         0.00
Kurtosis:                      19.892   Cond. No.                         313.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [78]:
y_backers = y_var['Backers']
model = sm.OLS(y_backers, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Backers   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                 -0.024
Method:                 Least Squares   F-statistic:                    0.5404
Date:                Wed, 22 Jan 2025   Prob (F-statistic):              0.776
Time:                        22:09:11   Log-Likelihood:                -842.89
No. Observations:                 117   AIC:                             1700.
Df Residuals:                     110   BIC:                             1719.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    33.5450    479.496      0.070      0.944    -916.704     983.794
creativity               28.4346     26.097      1.090      0.278     -23.284      80.153
future-orientation       -2.6930     25.378     -0.106      0.916     -52.986      47.600
self-confidence          35.9018     37.026      0.970      0.334     -37.476     109.279
enthusiasm              -30.9510     42.777     -0.724      0.471    -115.726      53.824
proactive-personality     9.9954     32.807      0.305      0.761     -55.020      75.011
trustworthiness         -13.2842     16.920     -0.785      0.434     -46.816      20.248
==============================================================================
Omnibus:                       50.801   Durbin-Watson:                   1.509
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              111.182
Skew:                           1.825   Prob(JB):                     7.20e-25
Kurtosis:                       6.080   Cond. No.                         313.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""